In [1]:
from sklearn.naive_bayes import MultinomialNB
import os
os.chdir('/Users/renalkakhan/Documents/GitHub/CS598_DLH_Project/')
from dataset.preprocessing.word2vec_embeddings_gen import Word2VecFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import FastTextFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import USEFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import GloVeFeatureGeneration
from sklearn.metrics import f1_score
import csv
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

In [2]:
class NaiveBayes:
    def __init__(self, x_train, y_train, x_test, y_test):
        self.nb = MultinomialNB()
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        # print(self.x_train.shape, self.y_train.shape, self.x_test.shape, self.y_test.shape)
        
    def train(self):
        self.nb.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.nb.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        #print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
        return f1_macro, f1_micro

In [3]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']

In [4]:
column_headings = ["Morbidity Class", "NB_Macro F1", "NB_Micro F1"]

with open("./results/word-embeddings-features/performance_NB_W2V.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    

In [5]:
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = Word2VecFeatureGeneration(train_preprocessed_df, morbidity).word2vec_matrix_gen()
    X = np.abs(np.average(X, axis=1))
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list = []
    f1_micro_list = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training RF using TF-IDF Representation
        nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
        nb_obj.train()

        f1_macro, f1_micro = nb_obj.test_and_evaluate()

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)

    f1_macro = np.mean(f1_macro_list)
    f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_NB_W2V.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_NB_W2V.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

(572, 3570, 100) (572,) Counter({0.0: 502, 1.0: 70})
Macro F1 score: 0.4672986632039369 and Micro F1 Score 0.8776164549304294
(548, 3570, 100) (548,) Counter({1.0: 325, 0.0: 223})
Macro F1 score: 0.3690486290082774 and Micro F1 Score 0.5857239057239056
(243, 3122, 100) (243,) Counter({1.0: 243})
Macro F1 score: 1.0 and Micro F1 Score 1.0
(582, 3570, 100) (582,) Counter({0.0: 460, 1.0: 122})
Macro F1 score: 0.4408968801610552 and Micro F1 Score 0.790385739333723
(567, 3570, 100) (567,) Counter({1.0: 396, 0.0: 171})
Macro F1 score: 0.409804351439744 and Micro F1 Score 0.6985902255639098
(593, 3570, 100) (593,) Counter({0.0: 506, 1.0: 87})
Macro F1 score: 0.4599902140879871 and Micro F1 Score 0.8532485875706215
(487, 3570, 100) (487,) Counter({0.0: 372, 1.0: 115})
Macro F1 score: 0.4323776782842145 and Micro F1 Score 0.7638605442176871
(596, 3570, 100) (596,) Counter({0.0: 518, 1.0: 78})
Macro F1 score: 0.4647957966912661 and Micro F1 Score 0.8691242937853108
(502, 3570, 100) (502,) Count

In [6]:
with open("./results/word-embeddings-features/performance_NB_Glove.csv", "w", newline="") as file:
    writer = csv.writer(file)

    # write the RF heading and the subheadings for Micro F1 and Macro F1
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])

all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = GloVeFeatureGeneration(train_preprocessed_df, morbidity).glove_matrix_gen()

    X = np.abs(np.average(X, axis=1))
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list = []
    f1_micro_list = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training RF using TF-IDF Representation
        nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
        nb_obj.train()

        f1_macro, f1_micro = nb_obj.test_and_evaluate()

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)

    f1_macro = np.mean(f1_macro_list)
    f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_NB_Glove.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_NB_Glove.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma


/Users/renalkakhan/Documents/GitHub/CS598_DLH_Project/dataset/preprocessing/word2vec_embeddings_gen.py:74: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X[i, :, :] = np.array(sentence_vectors)


ValueError: setting an array element with a sequence.

In [ ]:
with open("./results/word-embeddings-features/performance_NB_FastText.csv", "w", newline="") as file:
    writer = csv.writer(file)

    # write the RF heading and the subheadings for Micro F1 and Macro F1
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])

all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = FastTextFeatureGeneration(train_preprocessed_df, morbidity).fasttext_matrix_gen()

    X = np.abs(np.average(X, axis=1))
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list = []
    f1_micro_list = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training RF using TF-IDF Representation
        nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
        nb_obj.train()

        f1_macro, f1_micro = nb_obj.test_and_evaluate()

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)

    f1_macro = np.mean(f1_macro_list)
    f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_NB_FastText.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_NB_FastText.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
(572, 3570, 100) (572,) Counter({0.0: 502, 1.0: 70})
Macro F1 score: 0.4672986632039369 and Micro F1 Score 0.8776164549304294
CAD
(548, 3570, 100) (548,) Counter({1.0: 325, 0.0: 223})
Macro F1 score: 0.3762080985740671 and Micro F1 Score 0.5839057239057239
CHF
(243, 3122, 100) (243,) Counter({1.0: 243})
Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(582, 3570, 100) (582,) Counter({0.0: 460, 1.0: 122})
Macro F1 score: 0.4404041856316634 and Micro F1 Score 0.7886616014026885
Diabetes
(567, 3570, 100) (567,) Counter({1.0: 396, 0.0: 171})
Macro F1 score: 0.409804351439744 and Micro F1 Score 0.6985902255639098
Gallstones
(593, 3570, 100) (593,) Counter({0.0: 506, 1.0: 87})
Macro F1 score: 0.4599902140879871 and Micro F1 Score 0.8532485875706215
GERD
(487, 3570, 100) (487,) Counter({0.0: 372, 1.0: 115})
Macro F1 score: 0.4323776782842145 and Micro F1 Score 0.7638605442176871
Gout
(596, 3570, 100) (596,) Counter({0.0: 518, 1.0: 78})
Macro F1 score: 0.4738123194692568 and Micro 

In [ ]:
with open("./results/word-embeddings-features/performance_NB_USE.csv", "w", newline="") as file:
    writer = csv.writer(file)

    # write the RF heading and the subheadings for Micro F1 and Macro F1
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])

all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = USEFeatureGeneration(train_preprocessed_df, morbidity).use_matrix_gen()

    X = np.abs(np.average(X, axis=1))
    X = X.reshape(-1, 1)
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list = []
    f1_micro_list = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training RF using TF-IDF Representation
        nb_obj = NaiveBayes(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
        nb_obj.train()

        f1_macro, f1_micro = nb_obj.test_and_evaluate()

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)

    f1_macro = np.mean(f1_macro_list)
    f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_NB_USE.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_NB_USE.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma


2023-04-30 20:03:03.874065: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


(572, 512) (572,) Counter({0.0: 502, 1.0: 70})
Macro F1 score: 0.4672986632039369 and Micro F1 Score 0.8776164549304294
CAD
(548, 512) (548,) Counter({1.0: 325, 0.0: 223})
Macro F1 score: 0.37159345260573257 and Micro F1 Score 0.592996632996633
CHF
(243, 512) (243,) Counter({1.0: 243})
Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(582, 512) (582,) Counter({0.0: 460, 1.0: 122})
Macro F1 score: 0.4408968801610552 and Micro F1 Score 0.790385739333723
Diabetes
(567, 512) (567,) Counter({1.0: 396, 0.0: 171})
Macro F1 score: 0.409804351439744 and Micro F1 Score 0.6985902255639098
Gallstones
(593, 512) (593,) Counter({0.0: 506, 1.0: 87})
Macro F1 score: 0.4599902140879871 and Micro F1 Score 0.8532485875706215
GERD
(487, 512) (487,) Counter({0.0: 372, 1.0: 115})
Macro F1 score: 0.4323776782842145 and Micro F1 Score 0.7638605442176871
Gout
(596, 512) (596,) Counter({0.0: 518, 1.0: 78})
Macro F1 score: 0.4647957966912661 and Micro F1 Score 0.8691242937853108
Hypercholesterolemia
(502, 5